# Question 1: How can a traditional - non neural network approach be used to classify celestial objects (Stars, Galaxies, and Quasars)

## Imports

In [15]:
import pandas as pd

## Initialising the dataset

I am using the SDSS DR14 dataset obtained from Kaggle: https://www.kaggle.com/datasets/lucidlenn/sloan-digital-sky-survey/data.

In [14]:
# Import the dataset as from a csv file to a Pandas dataframe
path = '/Users/ryanu/Documents/Uni/ACT/SDSS-DR14-Classification/SDSS Data.csv'
data = pd.read_csv(path)

This dataset contains 18 columns, however most of them are not needed. I have chosen to only use the five filter bands; u, g, r, i, and z; and redshift.

In [17]:
parameters = data[["ra", "dec", "u", "g", "r", "i", "z", "redshift"]]
classification_data = data["class"]